In [1]:
import PyPDF2
import regex as re

In [78]:
# Request user to input the filename without the .pdf
filename = input('What is the file name?\n')

What is the file name?
D7


In [79]:
# Enter the sections to be extracted here
section_to_extract = [1,2,3,4,6,7,8,11,12,13,14,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]

## Getting the PDF File Text

This section of code will handle obtaining the file and the text within that pdf file.

In [80]:
# Open the File
file = open(f'{filename}.pdf','rb')

# Create PyPDF2 Reader Object
reader = PyPDF2.PdfFileReader(file)

# Get the Text from the file
text = ''

for i in range(reader.numPages):
    # Get the current page
    page = reader.getPage(i)
    # Extract the text
    current_text = page.extract_text()
    # Append to master text variable
    text += current_text

## Separate the text by Section

This section of code will separate the text by section and return a dictionary of section and the corresponding text.

In [81]:
# Section Regex
section_regex = re.escape(filename) + r'\.\d{1,2}\n'

# Create a Regex to Identify section numbers
sections_full = re.findall(section_regex,text)

# Split the text into an array
section_text = re.split(section_regex,text)

# Get a list of Sections
sections = [s.strip('\n') for s in sections_full]

# Remove the first section (Before D#.1)
section_text.pop(0)

# Create Dictionary
doc_dict = dict(zip(sections,section_text))

## Extract Target Data

This section of code will search for the sections specified by the user and return it as a string.

In [82]:
def remove_linespace(text):
    # Split by .\n to get an array
    # linespace_regex = r'(?<=[a-z)])\.\n(?=[A-Z(])'
    linespace_regex = r'[\.;:]\n'
    
    delimiter_arr = re.findall(linespace_regex,text)
    text_arr = re.split(linespace_regex,text)
    
    # Strip '\n'
    text_arr_formatted = [s.replace('\n','') for s in text_arr]
    
    # Join together
    full_text = ''
    
    for idx, text_formatted in enumerate(text_arr_formatted):
        if idx > 0:
            full_text += delimiter_arr[idx-1]
            full_text += text_formatted
        else:
            full_text += text_formatted
        
    return full_text

In [83]:
target_text = ''

for target in section_to_extract:
    section = f'{filename}.{target}'
    target_text += f'\n{section}\n'
    target_text += remove_linespace(doc_dict[section])

## Writing to Text File

This section will handle writing the data obtained into a text file.

In [84]:
with open(f'{filename}.txt','w') as text_file:
    text_file.write('%s' % target_text)